In [1]:
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node = sy.orchestra.launch(name="test", reset=True)

In [24]:
domain_client = node.login(email="info@openmined.org", password="changethis")

Logged into <test: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
@sy.syft_function_single_use()
def test_func(client, data):
    
    def process_batch(batch):
        import time
        time.sleep(1)
        return sum(batch)
    
    results = []
    for batch in data:
        pointer = client.launch_job(process_batch, batch)
        results.append(pointer)
    
    return sum([res.wait().get() for res in results])

SyftSuccess: Syft function 'test_func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
test_func(client=domain_client, data=[[1,2,3], [1,2,3], [1,2,3]])

18

In [18]:
import ast
from syft.service.code.code_parse import GlobalsVisitor
from syft.service.code.unparse import unparse
from typing import List
def process_code(
    raw_code: str,
    func_name: str,
    original_func_name: str,
    input_kwargs: List[str],
) -> str:
    tree = ast.parse(raw_code)

    # check there are no globals
    v = GlobalsVisitor()
    v.visit(tree)

    f = tree.body[0]
    f.decorator_list = []

    keywords = [ast.keyword(arg=i, value=[ast.Name(id=i)]) for i in input_kwargs]
    call_stmt = ast.Assign(
        targets=[ast.Name(id="result")],
        value=ast.Call(
            func=ast.Name(id=original_func_name), args=[], keywords=keywords
        ),
        lineno=0,
    )

    return_stmt = ast.Return(value=ast.Name(id="result"))
    new_body = tree.body + [call_stmt, return_stmt]

    wrapper_function = ast.FunctionDef(
        name=func_name,
        args=f.args,
        body=new_body,
        decorator_list=[],
        returns=None,
        lineno=0,
    )

    return unparse(wrapper_function), wrapper_function

In [19]:
from syft.client.api import NodeIdentity

input_kwargs = test_func.input_policy_init_kwargs
node_view_workaround = False
for k in input_kwargs.keys():
    if isinstance(k, NodeIdentity):
        node_view_workaround = True

if not node_view_workaround:
    input_keys = list(input_kwargs.keys())
else:
    input_keys = []
    for d in input_kwargs.values():
        input_keys += d.keys()
            
code_string, ast_obj = process_code(
    raw_code=test_func.code,
    func_name='final_test_func',
    original_func_name='test_func',
    input_kwargs=input_keys
)

In [16]:
print(code_string)

def final_test_func(client, data):

    def test_func(client, data):

        def process_batch(batch):
            import time
            time.sleep(1)
            return sum(batch)
        results = []
        for batch in data:
            pointer = client.launch_job(process_batch, batch)
            results.append(pointer)
        return sum([res.wait().get() for res in results])
    result = test_func()
    return result


In [ ]:
def wrapper(func, *args, **kwargs):
    # create user code
    # Question: We need action objects as inputs for input policy
    # for creating an UserCode, but inside the scope of a function
    # we assumed we have only data
    
    
    # submit user code
    # approve user code
    # run the user code async
    return

In [23]:
print(ast.dump(ast_obj, indent=2))

FunctionDef(
  name='final_test_func',
  args=arguments(
    posonlyargs=[],
    args=[
      arg(arg='client'),
      arg(arg='data')],
    kwonlyargs=[],
    kw_defaults=[],
    defaults=[]),
  body=[
    FunctionDef(
      name='test_func',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='client'),
          arg(arg='data')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        FunctionDef(
          name='process_batch',
          args=arguments(
            posonlyargs=[],
            args=[
              arg(arg='batch')],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]),
          body=[
            Import(
              names=[
                alias(name='time')]),
            Expr(
              value=Call(
                func=Attribute(
                  value=Name(id='time', ctx=Load()),
                  attr='sleep',
                  ctx=Load()),
                args=[
    